In [1]:
# importing required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.cluster import KMeans
from tqdm import tqdm
import time

train = pd.read_csv("CSV_train.csv",low_memory=False,delimiter=';')
test=pd.read_csv("CSV_test.csv",low_memory=False,delimiter=',')
hidden=pd.read_csv("CSV_hidden_test.csv",low_memory=False,delimiter=',')

In [2]:
#Storing dataset wells and depths
train_well = train.WELL.values; train_depth = train.DEPTH_MD.values
test_well = test.WELL.values; test_depth = test.DEPTH_MD.values
hidden_well = hidden.WELL.values; hidden_depth = hidden.DEPTH_MD.values

In [3]:
# storing length of datasets 
train_len = train.shape[0] 
test_len = test.shape[0]
All_data = pd.concat((train,test,hidden)).reset_index(drop=True) 

lithology_keys = {30000: 'Sandstone',
                 65030: 'Sandstone/Shale',
                 65000: 'Shale',
                 80000: 'Marl',
                 74000: 'Dolomite',
                 70000: 'Limestone',
                 70032: 'Chalk',
                 88000: 'Halite',
                 86000: 'Anhydrite',
                 99000: 'Tuff',
                 90000: 'Coal',
                 93000: 'Basement'}
All_data['Lithology'] = All_data['FORCE_2020_LITHOFACIES_LITHOLOGY'].map(lithology_keys)
All_data

,WELL,DEPTH_MD,X_LOC,Y_LOC,Z_LOC,GROUP,FORMATION,CALI,RSHA,RMED,...,DTS,DCAL,DRHO,MUDWEIGHT,RMIC,ROPA,RXO,FORCE_2020_LITHOFACIES_LITHOLOGY,FORCE_2020_LITHOFACIES_CONFIDENCE,Lithology
0,15/9-13,494.5280,437641.96875,6470972.5,-469.501831,NORDLAND GP.,NaN,19.480835,NaN,1.611410,...,NaN,NaN,-0.574928,NaN,NaN,NaN,NaN,65000,1.0,Shale
1,15/9-13,494.6800,437641.96875,6470972.5,-469.653809,NORDLAND GP.,NaN,19.468800,NaN,1.618070,...,NaN,NaN,-0.570188,NaN,NaN,NaN,NaN,65000,1.0,Shale
2,15/9-13,494.8320,437641.96875,6470972.5,-469.805786,NORDLAND GP.,NaN,19.468800,NaN,1.626459,...,NaN,NaN,-0.574245,NaN,NaN,NaN,NaN,65000,1.0,Shale
3,15/9-13,494.9840,437641.96875,6470972.5,-469.957794,NORDLAND GP.,NaN,19.459282,NaN,1.621594,...,NaN,NaN,-0.586315,NaN,NaN,NaN,NaN,65000,1.0,Shale
4,15/9-13,495.1360,437641.96875,6470972.5,-470.109772,NORDLAND GP.,NaN,19.453100,NaN,1.602679,...,NaN,NaN,-0.597914,NaN,NaN,NaN,NaN,65000,1.0,Shale
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1429689,35/9-7,2973.2988,536096.06250,6793022.0,-2943.444580,BAAT GP.,Etive Fm.,8.276272,NaN,2.820439,...,136.911575,NaN,0.502458,NaN,2.311106,24.306124,NaN,65000,2.0,Shale
1429690,35/9-7,2973.4508,536096.06250,6793022.0,-2943.595947,BAAT GP.,Etive Fm.,8.267273,NaN,3.020778,...,137.583923,NaN,0.374753,NaN,1.853418,22.201078,NaN,65000,2.0,Shale
1429691,35/9-7,2973.6028,536096.06250,6793022.0,-2943.747559,BAAT GP.,Etive Fm.,8.250099,NaN,2.795711,...,138.310898,NaN,0.211487,NaN,1.325961,20.096741,NaN,65000,2.0,Shale
1429692,35/9-7,2973.7548,536096.06250,6793022.0,-2943.899170,BAAT GP.,Etive Fm.,NaN,NaN,2.658694,...,137.592819,NaN,0.147950,NaN,1.260347,17.992323,NaN,65000,2.0,Shale


In [4]:
#dropping columns with high missing values
drop_cols = ['SGR', 'ROPA', 'RXO', 'MUDWEIGHT','DCAL','RMIC','FORCE_2020_LITHOFACIES_CONFIDENCE']
All_data_drop = All_data.drop(drop_cols, axis=1)
# encoding categorical variables
All_data_drop['GROUP_encoded'] = All_data_drop['GROUP'].astype('category')
All_data_drop['GROUP_encoded'] = All_data_drop['GROUP_encoded'].cat.codes

All_data_drop['FORMATION_encoded'] = All_data_drop['FORMATION'].astype('category')
All_data_drop['FORMATION_encoded'] = All_data_drop['FORMATION_encoded'].cat.codes

All_data_drop['WELL_encoded'] = All_data_drop['WELL'].astype('category')
All_data_drop['WELL_encoded'] = All_data_drop['WELL_encoded'].cat.codes

All_data_drop['Lithology_encoded'] = All_data_drop['FORCE_2020_LITHOFACIES_LITHOLOGY'].astype('category')
All_data_drop['Lithology_encoded'] = All_data_drop['Lithology_encoded'].cat.codes

In [5]:
All_data_drop

,WELL,DEPTH_MD,X_LOC,Y_LOC,Z_LOC,GROUP,FORMATION,CALI,RSHA,RMED,...,BS,ROP,DTS,DRHO,FORCE_2020_LITHOFACIES_LITHOLOGY,Lithology,GROUP_encoded,FORMATION_encoded,WELL_encoded,Lithology_encoded
0,15/9-13,494.5280,437641.96875,6470972.5,-469.501831,NORDLAND GP.,NaN,19.480835,NaN,1.611410,...,NaN,34.636410,NaN,-0.574928,65000,Shale,6,-1,1,1
1,15/9-13,494.6800,437641.96875,6470972.5,-469.653809,NORDLAND GP.,NaN,19.468800,NaN,1.618070,...,NaN,34.636410,NaN,-0.570188,65000,Shale,6,-1,1,1
2,15/9-13,494.8320,437641.96875,6470972.5,-469.805786,NORDLAND GP.,NaN,19.468800,NaN,1.626459,...,NaN,34.779556,NaN,-0.574245,65000,Shale,6,-1,1,1
3,15/9-13,494.9840,437641.96875,6470972.5,-469.957794,NORDLAND GP.,NaN,19.459282,NaN,1.621594,...,NaN,39.965164,NaN,-0.586315,65000,Shale,6,-1,1,1
4,15/9-13,495.1360,437641.96875,6470972.5,-470.109772,NORDLAND GP.,NaN,19.453100,NaN,1.602679,...,NaN,57.483765,NaN,-0.597914,65000,Shale,6,-1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1429689,35/9-7,2973.2988,536096.06250,6793022.0,-2943.444580,BAAT GP.,Etive Fm.,8.276272,NaN,2.820439,...,8.5,15.195305,136.911575,0.502458,65000,Shale,0,16,113,1
1429690,35/9-7,2973.4508,536096.06250,6793022.0,-2943.595947,BAAT GP.,Etive Fm.,8.267273,NaN,3.020778,...,8.5,15.770223,137.583923,0.374753,65000,Shale,0,16,113,1
1429691,35/9-7,2973.6028,536096.06250,6793022.0,-2943.747559,BAAT GP.,Etive Fm.,8.250099,NaN,2.795711,...,8.5,16.418465,138.310898,0.211487,65000,Shale,0,16,113,1
1429692,35/9-7,2973.7548,536096.06250,6793022.0,-2943.899170,BAAT GP.,Etive Fm.,NaN,NaN,2.658694,...,8.5,17.037945,137.592819,0.147950,65000,Shale,0,16,113,1


In [6]:
#dropping categorial features replaces beforehan by encoded features
drop2 = All_data_drop.drop(['GROUP', 'FORMATION','WELL','FORCE_2020_LITHOFACIES_LITHOLOGY','Lithology'], axis=1)

# splitting dataset into training, test, and hidden sets
train_prep = drop2[:train_len].copy()
test_prep = drop2[train_len:(train_len+test_len)].copy()
hidden_prep = drop2[(train_len+test_len):].copy()

In [7]:
drop2

,DEPTH_MD,X_LOC,Y_LOC,Z_LOC,CALI,RSHA,RMED,RDEP,RHOB,GR,...,DTC,SP,BS,ROP,DTS,DRHO,GROUP_encoded,FORMATION_encoded,WELL_encoded,Lithology_encoded
0,494.5280,437641.96875,6470972.5,-469.501831,19.480835,NaN,1.611410,1.798681,1.884186,80.200851,...,161.131180,24.612379,NaN,34.636410,NaN,-0.574928,6,-1,1,1
1,494.6800,437641.96875,6470972.5,-469.653809,19.468800,NaN,1.618070,1.795641,1.889794,79.262886,...,160.603470,23.895531,NaN,34.636410,NaN,-0.570188,6,-1,1,1
2,494.8320,437641.96875,6470972.5,-469.805786,19.468800,NaN,1.626459,1.800733,1.896523,74.821999,...,160.173615,23.916357,NaN,34.779556,NaN,-0.574245,6,-1,1,1
3,494.9840,437641.96875,6470972.5,-469.957794,19.459282,NaN,1.621594,1.801517,1.891913,72.878922,...,160.149429,23.793688,NaN,39.965164,NaN,-0.586315,6,-1,1,1
4,495.1360,437641.96875,6470972.5,-470.109772,19.453100,NaN,1.602679,1.795299,1.880034,71.729141,...,160.128342,24.104078,NaN,57.483765,NaN,-0.597914,6,-1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1429689,2973.2988,536096.06250,6793022.0,-2943.444580,8.276272,NaN,2.820439,3.158570,NaN,90.720284,...,75.260658,NaN,8.5,15.195305,136.911575,0.502458,0,16,113,1
1429690,2973.4508,536096.06250,6793022.0,-2943.595947,8.267273,NaN,3.020778,3.332977,NaN,87.062027,...,74.868301,NaN,8.5,15.770223,137.583923,0.374753,0,16,113,1
1429691,2973.6028,536096.06250,6793022.0,-2943.747559,8.250099,NaN,2.795711,3.044179,NaN,86.115921,...,74.848122,NaN,8.5,16.418465,138.310898,0.211487,0,16,113,1
1429692,2973.7548,536096.06250,6793022.0,-2943.899170,NaN,NaN,2.658694,2.847681,NaN,89.497131,...,74.964027,NaN,8.5,17.037945,137.592819,0.147950,0,16,113,1


In [8]:
train_prep1= train_prep.copy()
test_prep1= test_prep.copy()
hidden_prep1= hidden_prep.copy()

In [9]:
#Inputing missing values by introducing median 
from sklearn.impute import SimpleImputer
miss = SimpleImputer(missing_values=np.nan, strategy='median')
miss.fit(drop2)
All_imp = miss.fit_transform(drop2)
All_imp=pd.DataFrame(All_imp, columns=['DEPTH_MD', 'X_LOC', 'Y_LOC', 'Z_LOC', 'CALI', 'RSHA', 'RMED', 'RDEP',
       'RHOB', 'GR', 'NPHI', 'PEF', 'DTC', 'SP', 'BS', 'ROP', 'DTS', 'DRHO',
         'GROUP_encoded',
       'FORMATION_encoded', 'WELL_encoded','Lithology_encoded'])
All_imp

,DEPTH_MD,X_LOC,Y_LOC,Z_LOC,CALI,RSHA,RMED,RDEP,RHOB,GR,...,DTC,SP,BS,ROP,DTS,DRHO,GROUP_encoded,FORMATION_encoded,WELL_encoded,Lithology_encoded
0,494.5280,437641.96875,6470972.5,-469.501831,19.480835,1.398049,1.611410,1.798681,1.884186,80.200851,...,161.131180,24.612379,12.250001,34.636410,189.362198,-0.574928,6.0,-1.0,1.0,1.0
1,494.6800,437641.96875,6470972.5,-469.653809,19.468800,1.398049,1.618070,1.795641,1.889794,79.262886,...,160.603470,23.895531,12.250001,34.636410,189.362198,-0.570188,6.0,-1.0,1.0,1.0
2,494.8320,437641.96875,6470972.5,-469.805786,19.468800,1.398049,1.626459,1.800733,1.896523,74.821999,...,160.173615,23.916357,12.250001,34.779556,189.362198,-0.574245,6.0,-1.0,1.0,1.0
3,494.9840,437641.96875,6470972.5,-469.957794,19.459282,1.398049,1.621594,1.801517,1.891913,72.878922,...,160.149429,23.793688,12.250001,39.965164,189.362198,-0.586315,6.0,-1.0,1.0,1.0
4,495.1360,437641.96875,6470972.5,-470.109772,19.453100,1.398049,1.602679,1.795299,1.880034,71.729141,...,160.128342,24.104078,12.250001,57.483765,189.362198,-0.597914,6.0,-1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1429689,2973.2988,536096.06250,6793022.0,-2943.444580,8.276272,1.398049,2.820439,3.158570,2.331407,90.720284,...,75.260658,54.270451,8.500000,15.195305,136.911575,0.502458,0.0,16.0,113.0,1.0
1429690,2973.4508,536096.06250,6793022.0,-2943.595947,8.267273,1.398049,3.020778,3.332977,2.331407,87.062027,...,74.868301,54.270451,8.500000,15.770223,137.583923,0.374753,0.0,16.0,113.0,1.0
1429691,2973.6028,536096.06250,6793022.0,-2943.747559,8.250099,1.398049,2.795711,3.044179,2.331407,86.115921,...,74.848122,54.270451,8.500000,16.418465,138.310898,0.211487,0.0,16.0,113.0,1.0
1429692,2973.7548,536096.06250,6793022.0,-2943.899170,12.515673,1.398049,2.658694,2.847681,2.331407,89.497131,...,74.964027,54.270451,8.500000,17.037945,137.592819,0.147950,0.0,16.0,113.0,1.0


In [10]:
train_imp = All_imp[:train_len].copy()
test_imp = All_imp[train_len:(train_len+test_len)].copy()
hidden_imp = All_imp[(train_len+test_len):].copy()

In [11]:
print(train_imp.shape)
print(test_imp.shape)
print(hidden_imp.shape)

(1170511, 22)
(136786, 22)
(122397, 22)


In [12]:
from sklearn.preprocessing import StandardScaler, Normalizer, MinMaxScaler
x_header=['DEPTH_MD', 'X_LOC', 'Y_LOC', 'Z_LOC', 'CALI', 'RSHA', 'RMED', 'RDEP',
       'RHOB', 'GR', 'NPHI', 'PEF', 'DTC', 'SP', 'BS', 'ROP', 'DTS', 'DRHO',
       'GROUP_encoded', 'FORMATION_encoded', 'WELL_encoded']
y_header=['Lithology_encoded']
x_train = train_imp[x_header]
y_train = train_imp[y_header]
x_test = test_imp[x_header]
y_test = test_imp[y_header]
x_hidden = hidden_imp[x_header]
y_hidden = hidden_imp[y_header]

##Min-Max scaler 
scaler = MinMaxScaler()
x_train_scaled = x_train.copy()
x_test_scaled = x_test.copy()
x_hidden_scaled = x_hidden.copy()

x_train_scaled.iloc[:,:18] = scaler.fit_transform(x_train_scaled.iloc[:,:18])
x_test_scaled.iloc[:,:18] = scaler.transform(x_test_scaled.iloc[:,:18])
x_hidden_scaled.iloc[:,:18] = scaler.transform(x_hidden_scaled.iloc[:,:18])

In [13]:
def augment_features_window(X, N_neig):
    
    # Parameters
    N_row = X.shape[0]
    N_feat = X.shape[1]

    # Zero padding
    X = np.vstack((np.zeros((N_neig, N_feat)), X, (np.zeros((N_neig, N_feat)))))

    # Loop over windows
    X_aug = np.zeros((N_row, N_feat*(2*N_neig+1)))
    for r in np.arange(N_row)+N_neig:
        this_row = []
        for c in np.arange(-N_neig,N_neig+1):
            this_row = np.hstack((this_row, X[r+c]))
        X_aug[r-N_neig] = this_row

    return X_aug

# Feature gradient computation function
def augment_features_gradient(X, depth):
    
    # Compute features gradient
    d_diff = np.diff(depth).reshape((-1, 1))
    d_diff[d_diff==0] = 0.001
    X_diff = np.diff(X, axis=0)
    X_grad = X_diff / d_diff
        
    # Compensate for last missing value
    X_grad = np.concatenate((X_grad, np.zeros((1, X_grad.shape[1]))))
    
    return X_grad

#Feature augmentation function
def augment_features(X, well, depth, N_neig=1):
    
    # Augment features
    X_aug = np.zeros((X.shape[0], X.shape[1]*(N_neig*2+2)))
    for w in np.unique(well):
        w_idx = np.where(well == w)[0]
        X_aug_win = augment_features_window(X[w_idx, :], N_neig)
        X_aug_grad = augment_features_gradient(X[w_idx, :], depth[w_idx])
        X_aug[w_idx, :] = np.concatenate((X_aug_win, X_aug_grad), axis=1)
    
    # Find padded rows
    padded_rows = np.unique(np.where(X_aug[:, 0:7] == np.zeros((1, 7)))[0])
    
    return X_aug, padded_rows

In [14]:
x_train_arg, padded_rows = augment_features(x_train_scaled.values, train_well, train_depth)
x_test_arg, padded_rows = augment_features(x_test_scaled.values, test_well, test_depth)
x_hidden_arg, padded_rows = augment_features(x_hidden_scaled.values, hidden_well, hidden_depth)

In [15]:
print('The shape of the TRAINING DATA before argumentation is {} and its shape after argumentation is {}'.format(x_train_scaled.shape, x_train_arg.shape))
print('The shape of the TEST DATA before argumentation is {} and its shape after argumentation is {}'.format(x_test_scaled.shape, x_test_arg.shape))
print('The shape of the HIDDEN DATA before argumentation is {} and its shape after argumentation is {}'.format(x_hidden_scaled.shape, x_hidden_arg.shape))

The shape of the TRAINING DATA before argumentation is (1170511, 21) and its shape after argumentation is (1170511, 84)
The shape of the TEST DATA before argumentation is (136786, 21) and its shape after argumentation is (136786, 84)
The shape of the HIDDEN DATA before argumentation is (122397, 21) and its shape after argumentation is (122397, 84)


In [23]:
#Supervised Algorithms
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import mean_squared_error, accuracy_score, recall_score, precision_score, f1_score
from sklearn.neighbors import KNeighborsRegressor
from pprint import pprint
from sklearn.model_selection import StratifiedKFold
from sklearn.naive_bayes import GaussianNB
import xgboost
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
#Comparing base models accuracies by using k-fold cross validation - 10 folds

from sklearn.model_selection import cross_val_score
x_train_arg1=pd.DataFrame(data=x_train_arg[0:,0:],index=[i for i in range(x_train_arg.shape[0])],columns=['f'+str(i) for i in range(x_train_arg.shape[1])])
x_train_arg1.head()

# x_train_arg = pd.DataFrame(x_train_arg, columns = x_train_arg.columns)

new_train = pd.concat((x_train_arg1, pd.DataFrame(y_train, columns=["Lithology_encoded"])), axis=1)

#Randomly sampling data
# x_train_sam = x_train_arg.sample(n=100000, random_state=0)
# y_train_sam= y_train.sample(n=100000, random_state=0)

sampled_train = new_train.sample(n=100000, random_state=0)


# #Spliting training data
x_train_sam = sampled_train.drop(["Lithology_encoded"], axis=1)
y_train_sam = sampled_train["Lithology_encoded"]



In [25]:
x_train_sam

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f74,f75,f76,f77,f78,f79,f80,f81,f82,f83
131073,0.329934,0.342195,0.259872,0.669321,0.243575,0.000637,0.000915,0.000937,0.606198,0.076329,...,-0.001937,0.023721,0.000000,0.0,0.000070,0.048193,-3.262843e-06,0.0,0.0,0.0
352995,0.262464,0.298382,0.447807,0.736780,0.392612,0.000637,0.000455,0.000394,0.444301,0.056614,...,-0.002480,-0.023976,0.000365,0.0,0.000000,0.000000,-2.057335e-06,0.0,0.0,0.0
1010257,0.428335,0.642723,0.823767,0.570190,0.392199,0.000637,0.000632,0.000654,0.588427,0.150571,...,0.000000,-0.001960,0.000000,0.0,0.000047,0.000000,0.000000e+00,0.0,0.0,0.0
1088778,0.673290,0.612037,0.923949,0.324952,0.380858,0.000637,0.000824,0.000794,0.708912,0.060377,...,-0.017127,0.002998,0.000000,0.0,0.000000,-0.003129,-4.843376e-06,0.0,0.0,0.0
994174,0.493365,0.710079,0.818927,0.504974,0.243747,0.000637,0.008675,0.007525,0.684489,0.095327,...,0.000405,0.013093,0.000938,0.0,0.000000,-0.017231,-1.166780e-06,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
283610,0.601850,0.363341,0.520981,0.396447,0.285717,0.000530,0.000588,0.000446,0.594447,0.100140,...,-0.004471,0.022617,0.001808,0.0,0.000000,0.000000,-7.141588e-07,0.0,0.0,0.0
1058390,0.179091,0.856816,0.838262,0.820565,0.579553,0.000160,0.000257,0.000580,0.490678,0.078622,...,-0.001845,0.007639,-0.000974,0.0,0.000000,0.000000,6.726667e-06,0.0,0.0,0.0
248801,0.246285,0.343299,0.486188,0.752482,0.420032,0.000674,0.000624,0.000686,0.508097,0.053964,...,-0.004131,-0.003495,0.000572,0.0,0.000000,0.000000,-4.412294e-06,0.0,0.0,0.0
727023,0.422181,0.074667,0.855368,0.576536,0.395274,0.000637,0.000806,0.000773,0.589713,0.044465,...,0.000000,0.024294,0.002045,0.0,0.000000,0.000000,6.812943e-06,0.0,0.0,0.0


In [26]:
estimator = LogisticRegression(C=1e-3, solver='saga', max_iter=4000)
score = cross_val_score(estimator, x_train_sam, y_train_sam.values.ravel(), cv=10, scoring='f1_weighted').mean()
score

0.483824472100492

In [27]:
estimator1 = DecisionTreeClassifier()
score = cross_val_score(estimator1, x_train_sam, y_train_sam.values.ravel(), cv=10, scoring='f1_weighted').mean()
score

0.8510715136906443

In [28]:
estimator2 = RandomForestClassifier()
score = cross_val_score(estimator2, x_train_sam, y_train_sam.values.ravel(), cv=10, scoring='f1_weighted').mean()
score

0.9115785817630316

In [29]:
estimator3 = XGBClassifier()
score3 = cross_val_score(estimator3, x_train_sam, y_train_sam.values.ravel(), cv=10, scoring='f1_weighted').mean()
score3

C:\Users\Ayori\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\Ayori\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[02:45:50] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\Ayori\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\Ayori\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[02:51:51] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\Ayori\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\Ayori\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


...

C:\Users\Ayori\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\Ayori\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[03:33:43] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\Ayori\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\Ayori\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[03:39:40] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


0.8940081240321505

In [30]:
estimator4 = GradientBoostingClassifier()
score4 = cross_val_score(estimator4, x_train_sam, y_train_sam.values.ravel(), cv=10, scoring='f1_weighted').mean()
score4


0.8360024956620107

In [31]:
from sklearn.neighbors import KNeighborsClassifier
estimator5 =KNeighborsClassifier()
score5 = cross_val_score(estimator5, x_train_sam, y_train_sam.values.ravel(), cv=10, scoring='f1_weighted').mean()
score5


0.8671973711029581

In [32]:
from sklearn.neighbors import KNeighborsClassifier
estimator6 =SVC()
score6 = cross_val_score(estimator6, x_train_sam, y_train_sam.values.ravel(), cv=10, scoring='f1_weighted').mean()
score6


0.47555592464801527

In [33]:
from catboost import CatBoostClassifier
estimator7 =CatBoostClassifier()
score7 = cross_val_score(estimator7, x_train_sam, y_train_sam.values.ravel(), cv=10, scoring='f1_weighted').mean()
score7


Learning rate set to 0.099538
0:	learn: 1.9496663	total: 989ms	remaining: 16m 28s
1:	learn: 1.7141519	total: 1.92s	remaining: 15m 58s
2:	learn: 1.5524051	total: 2.82s	remaining: 15m 38s
3:	learn: 1.4283852	total: 3.73s	remaining: 15m 27s
4:	learn: 1.3317327	total: 4.72s	remaining: 15m 38s
5:	learn: 1.2503943	total: 5.67s	remaining: 15m 39s
6:	learn: 1.1836810	total: 6.59s	remaining: 15m 35s
7:	learn: 1.1308616	total: 7.51s	remaining: 15m 31s
8:	learn: 1.0826992	total: 8.44s	remaining: 15m 28s
9:	learn: 1.0363174	total: 9.37s	remaining: 15m 27s
10:	learn: 0.9978914	total: 10.3s	remaining: 15m 27s
11:	learn: 0.9623199	total: 11.2s	remaining: 15m 25s
12:	learn: 0.9334026	total: 12.1s	remaining: 15m 22s
13:	learn: 0.9082426	total: 13.1s	remaining: 15m 19s
14:	learn: 0.8754107	total: 14s	remaining: 15m 17s
15:	learn: 0.8544823	total: 14.9s	remaining: 15m 14s
16:	learn: 0.8348203	total: 15.8s	remaining: 15m 11s
17:	learn: 0.8176934	total: 16.6s	remaining: 15m 8s
18:	learn: 0.7988858	total: 1

153:	learn: 0.4300453	total: 2m 14s	remaining: 12m 20s
154:	learn: 0.4294467	total: 2m 15s	remaining: 12m 19s
155:	learn: 0.4285154	total: 2m 16s	remaining: 12m 18s
156:	learn: 0.4273330	total: 2m 17s	remaining: 12m 17s
157:	learn: 0.4265477	total: 2m 18s	remaining: 12m 16s
158:	learn: 0.4259071	total: 2m 19s	remaining: 12m 15s
159:	learn: 0.4253128	total: 2m 19s	remaining: 12m 14s
160:	learn: 0.4245020	total: 2m 20s	remaining: 12m 13s
161:	learn: 0.4234710	total: 2m 21s	remaining: 12m 12s
162:	learn: 0.4229375	total: 2m 22s	remaining: 12m 11s
163:	learn: 0.4222688	total: 2m 23s	remaining: 12m 10s
164:	learn: 0.4214176	total: 2m 24s	remaining: 12m 9s
165:	learn: 0.4208114	total: 2m 25s	remaining: 12m 8s
166:	learn: 0.4201132	total: 2m 25s	remaining: 12m 7s
167:	learn: 0.4191965	total: 2m 26s	remaining: 12m 6s
168:	learn: 0.4182710	total: 2m 27s	remaining: 12m 5s
169:	learn: 0.4175872	total: 2m 28s	remaining: 12m 4s
170:	learn: 0.4163430	total: 2m 29s	remaining: 12m 3s
171:	learn: 0.415

304:	learn: 0.3557610	total: 4m 24s	remaining: 10m 1s
305:	learn: 0.3553061	total: 4m 24s	remaining: 10m
306:	learn: 0.3551214	total: 4m 25s	remaining: 9m 59s
307:	learn: 0.3548950	total: 4m 26s	remaining: 9m 58s
308:	learn: 0.3547157	total: 4m 27s	remaining: 9m 58s
309:	learn: 0.3545457	total: 4m 28s	remaining: 9m 57s
310:	learn: 0.3543836	total: 4m 29s	remaining: 9m 56s
311:	learn: 0.3541165	total: 4m 30s	remaining: 9m 55s
312:	learn: 0.3539336	total: 4m 30s	remaining: 9m 54s
313:	learn: 0.3536099	total: 4m 31s	remaining: 9m 53s
314:	learn: 0.3534143	total: 4m 32s	remaining: 9m 52s
315:	learn: 0.3532023	total: 4m 33s	remaining: 9m 51s
316:	learn: 0.3530146	total: 4m 34s	remaining: 9m 51s
317:	learn: 0.3528278	total: 4m 35s	remaining: 9m 50s
318:	learn: 0.3524099	total: 4m 36s	remaining: 9m 49s
319:	learn: 0.3520300	total: 4m 36s	remaining: 9m 48s
320:	learn: 0.3518337	total: 4m 37s	remaining: 9m 47s
321:	learn: 0.3515487	total: 4m 38s	remaining: 9m 46s
322:	learn: 0.3511658	total: 4m

457:	learn: 0.3217818	total: 6m 31s	remaining: 7m 43s
458:	learn: 0.3216313	total: 6m 32s	remaining: 7m 42s
459:	learn: 0.3213992	total: 6m 32s	remaining: 7m 41s
460:	learn: 0.3211727	total: 6m 33s	remaining: 7m 40s
461:	learn: 0.3210903	total: 6m 34s	remaining: 7m 39s
462:	learn: 0.3209111	total: 6m 35s	remaining: 7m 38s
463:	learn: 0.3208448	total: 6m 36s	remaining: 7m 37s
464:	learn: 0.3207353	total: 6m 36s	remaining: 7m 36s
465:	learn: 0.3205420	total: 6m 37s	remaining: 7m 35s
466:	learn: 0.3203788	total: 6m 38s	remaining: 7m 34s
467:	learn: 0.3200049	total: 6m 39s	remaining: 7m 33s
468:	learn: 0.3197047	total: 6m 39s	remaining: 7m 32s
469:	learn: 0.3194119	total: 6m 40s	remaining: 7m 31s
470:	learn: 0.3193692	total: 6m 41s	remaining: 7m 30s
471:	learn: 0.3191602	total: 6m 42s	remaining: 7m 29s
472:	learn: 0.3191098	total: 6m 42s	remaining: 7m 28s
473:	learn: 0.3190086	total: 6m 43s	remaining: 7m 28s
474:	learn: 0.3188214	total: 6m 44s	remaining: 7m 27s
475:	learn: 0.3187400	total:

610:	learn: 0.2973324	total: 8m 30s	remaining: 5m 24s
611:	learn: 0.2972400	total: 8m 31s	remaining: 5m 24s
612:	learn: 0.2971431	total: 8m 31s	remaining: 5m 23s
613:	learn: 0.2969345	total: 8m 32s	remaining: 5m 22s
614:	learn: 0.2968045	total: 8m 33s	remaining: 5m 21s
615:	learn: 0.2966208	total: 8m 34s	remaining: 5m 20s
616:	learn: 0.2963733	total: 8m 35s	remaining: 5m 19s
617:	learn: 0.2962444	total: 8m 35s	remaining: 5m 18s
618:	learn: 0.2961398	total: 8m 36s	remaining: 5m 17s
619:	learn: 0.2960472	total: 8m 37s	remaining: 5m 17s
620:	learn: 0.2959506	total: 8m 38s	remaining: 5m 16s
621:	learn: 0.2957260	total: 8m 38s	remaining: 5m 15s
622:	learn: 0.2955955	total: 8m 39s	remaining: 5m 14s
623:	learn: 0.2954887	total: 8m 40s	remaining: 5m 13s
624:	learn: 0.2953576	total: 8m 41s	remaining: 5m 12s
625:	learn: 0.2952563	total: 8m 41s	remaining: 5m 11s
626:	learn: 0.2951435	total: 8m 42s	remaining: 5m 10s
627:	learn: 0.2949398	total: 8m 43s	remaining: 5m 10s
628:	learn: 0.2948146	total:

...

457:	learn: 0.3242799	total: 6m 3s	remaining: 7m 10s
458:	learn: 0.3240899	total: 6m 4s	remaining: 7m 9s
459:	learn: 0.3239366	total: 6m 5s	remaining: 7m 8s
460:	learn: 0.3237897	total: 6m 6s	remaining: 7m 8s
461:	learn: 0.3236337	total: 6m 6s	remaining: 7m 7s
462:	learn: 0.3233258	total: 6m 7s	remaining: 7m 6s
463:	learn: 0.3231960	total: 6m 8s	remaining: 7m 5s
464:	learn: 0.3230879	total: 6m 9s	remaining: 7m 4s
465:	learn: 0.3227701	total: 6m 10s	remaining: 7m 4s
466:	learn: 0.3227125	total: 6m 10s	remaining: 7m 3s
467:	learn: 0.3226377	total: 6m 11s	remaining: 7m 2s
468:	learn: 0.3224712	total: 6m 12s	remaining: 7m 1s
469:	learn: 0.3223339	total: 6m 13s	remaining: 7m
470:	learn: 0.3221238	total: 6m 13s	remaining: 6m 59s
471:	learn: 0.3220460	total: 6m 14s	remaining: 6m 59s
472:	learn: 0.3217795	total: 6m 15s	remaining: 6m 58s
473:	learn: 0.3216559	total: 6m 16s	remaining: 6m 57s
474:	learn: 0.3215631	total: 6m 17s	remaining: 6m 56s
475:	learn: 0.3214992	total: 6m 17s	remaining: 6m 5

610:	learn: 0.2980325	total: 8m 3s	remaining: 5m 7s
611:	learn: 0.2977335	total: 8m 4s	remaining: 5m 7s
612:	learn: 0.2974126	total: 8m 5s	remaining: 5m 6s
613:	learn: 0.2973334	total: 8m 6s	remaining: 5m 5s
614:	learn: 0.2971153	total: 8m 6s	remaining: 5m 4s
615:	learn: 0.2970639	total: 8m 7s	remaining: 5m 4s
616:	learn: 0.2969168	total: 8m 8s	remaining: 5m 3s
617:	learn: 0.2967588	total: 8m 9s	remaining: 5m 2s
618:	learn: 0.2966442	total: 8m 10s	remaining: 5m 1s
619:	learn: 0.2965195	total: 8m 10s	remaining: 5m
620:	learn: 0.2961932	total: 8m 11s	remaining: 5m
621:	learn: 0.2961298	total: 8m 12s	remaining: 4m 59s
622:	learn: 0.2959686	total: 8m 13s	remaining: 4m 58s
623:	learn: 0.2957927	total: 8m 13s	remaining: 4m 57s
624:	learn: 0.2957464	total: 8m 14s	remaining: 4m 56s
625:	learn: 0.2956996	total: 8m 15s	remaining: 4m 56s
626:	learn: 0.2955415	total: 8m 16s	remaining: 4m 55s
627:	learn: 0.2954594	total: 8m 17s	remaining: 4m 54s
628:	learn: 0.2952419	total: 8m 17s	remaining: 4m 53s

763:	learn: 0.2777294	total: 10m 4s	remaining: 3m 6s
764:	learn: 0.2776107	total: 10m 5s	remaining: 3m 5s
765:	learn: 0.2775037	total: 10m 6s	remaining: 3m 5s
766:	learn: 0.2773796	total: 10m 7s	remaining: 3m 4s
767:	learn: 0.2772914	total: 10m 7s	remaining: 3m 3s
768:	learn: 0.2772291	total: 10m 8s	remaining: 3m 2s
769:	learn: 0.2770361	total: 10m 9s	remaining: 3m 2s
770:	learn: 0.2769670	total: 10m 10s	remaining: 3m 1s
771:	learn: 0.2768114	total: 10m 10s	remaining: 3m
772:	learn: 0.2766328	total: 10m 11s	remaining: 2m 59s
773:	learn: 0.2765386	total: 10m 12s	remaining: 2m 58s
774:	learn: 0.2764074	total: 10m 13s	remaining: 2m 58s
775:	learn: 0.2762507	total: 10m 14s	remaining: 2m 57s
776:	learn: 0.2760896	total: 10m 14s	remaining: 2m 56s
777:	learn: 0.2759808	total: 10m 15s	remaining: 2m 55s
778:	learn: 0.2758175	total: 10m 16s	remaining: 2m 54s
779:	learn: 0.2757562	total: 10m 17s	remaining: 2m 54s
780:	learn: 0.2756363	total: 10m 18s	remaining: 2m 53s
781:	learn: 0.2755453	total: 

914:	learn: 0.2623418	total: 12m 3s	remaining: 1m 7s
915:	learn: 0.2622948	total: 12m 4s	remaining: 1m 6s
916:	learn: 0.2622106	total: 12m 5s	remaining: 1m 5s
917:	learn: 0.2621159	total: 12m 6s	remaining: 1m 4s
918:	learn: 0.2620399	total: 12m 6s	remaining: 1m 4s
919:	learn: 0.2619896	total: 12m 7s	remaining: 1m 3s
920:	learn: 0.2616896	total: 12m 8s	remaining: 1m 2s
921:	learn: 0.2615820	total: 12m 9s	remaining: 1m 1s
922:	learn: 0.2614404	total: 12m 10s	remaining: 1m
923:	learn: 0.2613526	total: 12m 10s	remaining: 1m
924:	learn: 0.2612252	total: 12m 11s	remaining: 59.3s
925:	learn: 0.2611193	total: 12m 12s	remaining: 58.5s
926:	learn: 0.2610432	total: 12m 13s	remaining: 57.7s
927:	learn: 0.2609178	total: 12m 14s	remaining: 56.9s
928:	learn: 0.2608361	total: 12m 14s	remaining: 56.2s
929:	learn: 0.2606945	total: 12m 15s	remaining: 55.4s
930:	learn: 0.2605964	total: 12m 16s	remaining: 54.6s
931:	learn: 0.2604887	total: 12m 17s	remaining: 53.8s
932:	learn: 0.2602903	total: 12m 18s	remai

0.8841910036771555

In [34]:
from lightgbm import LGBMClassifier
estimator8 =LGBMClassifier()
score8 = cross_val_score(estimator8, x_train_sam, y_train_sam.values.ravel(), cv=10, scoring='f1_weighted').mean()
score8


0.7859591499511007

In [35]:
model_names = ['Logistic Regression', 'Decision Trees Classifier', 'Random Forest Classifier', 'XGB Classifier', 'Gradient Boosting Classifier', 'K-Nearest Neighbor Classifier', 'Support Vector Machines','CatBoost','LGBM']
f1_train_scores = [score, score,score,score3, score4, score5,score6,score7,score8]
base_models = pd.DataFrame(np.around(f1_train_scores,3), model_names, columns=['F1_SCORES']).sort_values('F1_SCORES', ascending=False)
base_models

,F1_SCORES
Logistic Regression,0.912
Decision Trees Classifier,0.912
Random Forest Classifier,0.912
XGB Classifier,0.894
CatBoost,0.884
K-Nearest Neighbor Classifier,0.867
Gradient Boosting Classifier,0.836
LGBM,0.786
Support Vector Machines,0.476
